In [36]:
# Библиотека, которая позволяет создавать модели нейронных сетей 
import tensorflow as tf
from sklearn.model_selection import train_test_split
# Позволяет создать массив фреймов аудиопотока 
import numpy as np
import os
# Связанные с numpy
from IPython.display import Audio
from IPython.core.display import HTML
# Библиотека, которая позволяет рисовать спектрограмму частоты звука от времени
# Работает с преобразованием Фурье 
from scipy.signal import spectrogram

In [37]:
# Для парсинга wav-файлов 
import scipy.io.wavfile as wav
# Мел частотные кепстральные коэффициенты (спектрограмма с мел-шкалой вместо оси у)
# Строит характеристику частот от времени в сэмплах через STFT 

# LSTM - для вызова рекурсивной функции
# Dense - для связи всех нейронов предыдущего слоя с текущим (выходным) слоем
# Convolution - сверта слоев
from keras.layers import LSTM, Dense, Convolution1D, Activation

# Позволяет давать послойное описание модели 
from keras.models import Sequential
# TimeDistributed - Один из способов работы с Dense 
# Bidirectional - получение информации не только от прошлого и самого себя, но от будущего  
from keras.layers.wrappers import TimeDistributed, Bidirectional
# Последовательность
from keras.utils import to_categorical

In [38]:
from python_speech_features import fbank, mfcc

## 1.  Загрузим аудио и тексты:

In [39]:
os.listdir('Voxforge')[0:10]

['1-20121125-pgp',
 '1981-20120705-haq',
 '1981-20120705-rjp',
 '1981-20120706-azq',
 '1981-20120706-hpa',
 '1981-20120706-kwo',
 '1981-20120706-rxa',
 '1981-20120706-rya',
 '1981-20120706-vmc',
 '1981-20120706-zfp']

In [40]:
os.getcwd()

'C:\\Users\\IQ-6\\Documents\\распознование речи'

In [41]:
path1 = os.getcwd()

In [42]:
vocabulary = { 'а': 1,
               'б': 2,
               'в': 3,
               'г': 4,
               'д': 5,
               'е': 6,
               'ё': 7,
               'ж': 8,
               'з': 9,
               'и': 10,
               'й': 11,
               'к': 12,
               'л': 13,
               'м': 14,
               'н': 15,
               'о': 16,
               'п': 17,
               'р': 18,
               'с': 19,
               'т': 20,
               'у': 21,
               'ф': 22,
               'х': 23,
               'ц': 24,
               'ч': 25,
               'ш': 26,
               'щ': 27,
               'ъ': 28,
               'ы': 29,
               'ь': 30,
               'э': 31,
               'ю': 32,
               'я': 33
             }
# Преобразование в словарь 
inv_mapping = dict(zip(vocabulary.values(), vocabulary.keys()))
inv_mapping[34]='<пробел>'

In [43]:
#path1 = os.getcwd()    
y = []
x = []
os.chdir(path1) #меняем директорию

for speaker_folder in os.listdir('Voxforge')[0:15]:
    
    
    
    speaker_folder_wav = 'Voxforge//'+speaker_folder+'//wav//'
    speaker_folder_txt = 'Voxforge//'+speaker_folder+'//txt//'
    
    for wav_file in os.listdir(speaker_folder_wav): 
        
        try:
            text_file = speaker_folder_txt + wav_file.split('.')[0] + '.txt'
       
            with open(text_file, 'rb') as f:
                for line in f.readlines():
                    if line[0] == ';':
                        continue
                


                original = ' '.join(str(line, 'utf-8').strip().lower().split(' ')).replace('.', '').replace("'", '').replace('-', '').replace(',','')
    # Меняем пробелы на более длинные 
                targets = original.replace(' ', '  ')
    # Разделяем слова через пробелы 
                targets = targets.split(' ')
    # Объявляем строку и начинаем ее с пробела
                stroka=[34]
                for i in targets:
                    i1=i.encode("UTF-8")
                    for j in range(0,len(i1),2):
    # Через вызов decode() получаем буквы и добавляем их в строку 
                        stroka.append(vocabulary.get(i1[j:j+2].decode("utf-8"),34))
                    if stroka[-1] != 34:
    # Закончили с буквами
    # Добавляем в конец строки пробел
                        stroka.append(34)
    # Обработка звука
                fs, audio = wav.read(speaker_folder_wav + wav_file)
    # Ищем отличительные черты через мел-кепстральные коэффициенты и преобразование Фурье            
    # Мел-шкала отражает главным образом высоту звука, от которой, в свою очередь, его частота. 
    # Эта зависимость нелинейна, особенно при низких частотах.
    # Различные звуки имеют различные частоты и, соответственно, по-разному отображаются
    # на мел-шкале.

# выделяет ту часть которая важна человеку 
                features = mfcc(audio, 
                                samplerate=fs,
                                winlen=0.2,   #размер окна 0,2 сек
                                winstep=0.1, # шаг 0,1 сек
                                numcep=16, # число мелов 16
                                lowfreq=50, #самая низкая чистота hg
                                nfft=4096 # число синусоид
                               )
                mean_scale = np.mean(features, axis=0) #нормировка
                std_scale = np.std(features, axis=0)

                features = (features - mean_scale[np.newaxis, :]) / std_scale[np.newaxis, :]
                seq_len = features.shape[0]
            

## возьмем только похожие по размеру, около 100 букв
#                 if seq_len > 950 and seq_len < 1050 and len (stroka) > 90 and len (stroka) < 110:
                
                x.append(features) #в x(массив) добавляем аудиозапись 
                y.append(stroka)  # текст в цифровом формате 
                print(wav_file,text_file ) # скачивание 
            
            
            
            
            
            
            
            
        except:
      #      
            print('Error ', speaker_folder, wav_file, text_file)
            continue

ru_0022.wav Voxforge//1-20121125-pgp//txt//ru_0022.txt
ru_0024.wav Voxforge//1-20121125-pgp//txt//ru_0024.txt
ru_0025.wav Voxforge//1-20121125-pgp//txt//ru_0025.txt
ru_0027.wav Voxforge//1-20121125-pgp//txt//ru_0027.txt
ru_0030.wav Voxforge//1-20121125-pgp//txt//ru_0030.txt
ru_0031.wav Voxforge//1-20121125-pgp//txt//ru_0031.txt
ru_0032.wav Voxforge//1-20121125-pgp//txt//ru_0032.txt
ru_0033.wav Voxforge//1-20121125-pgp//txt//ru_0033.txt
ru_0034.wav Voxforge//1-20121125-pgp//txt//ru_0034.txt
ru_0035.wav Voxforge//1-20121125-pgp//txt//ru_0035.txt
ru_0076.wav Voxforge//1981-20120705-haq//txt//ru_0076.txt
ru_0077.wav Voxforge//1981-20120705-haq//txt//ru_0077.txt
ru_0078.wav Voxforge//1981-20120705-haq//txt//ru_0078.txt
ru_0079.wav Voxforge//1981-20120705-haq//txt//ru_0079.txt
ru_0080.wav Voxforge//1981-20120705-haq//txt//ru_0080.txt
ru_0081.wav Voxforge//1981-20120705-haq//txt//ru_0081.txt
ru_0082.wav Voxforge//1981-20120705-haq//txt//ru_0082.txt
ru_0084.wav Voxforge//1981-20120705-haq//txt

ru_0035.wav Voxforge//akaalxcoaakaInterocitor-20100322-naz//txt//ru_0035.txt
ru_0067.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0067.txt
ru_0068.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0068.txt
ru_0069.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0069.txt
ru_0070.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0070.txt
ru_0071.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0071.txt
ru_0073.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0073.txt
ru_0074.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0074.txt
ru_0075.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0075.txt
ru_0076.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0076.txt
ru_0077.wav Voxforge//akaalxcoaakaInterocitor-20100324-hyp//txt//ru_0077.txt


In [44]:
os.getcwd()

'C:\\Users\\IQ-6\\Documents\\распознование речи'

In [45]:
# приведем к одной длине аудио записи, заполнив нулями?

lens_mfcc = []
for i in range(len(x)):
    lens_mfcc.append(x[i].shape[0])
    
print(max(lens_mfcc), min(lens_mfcc))

209 63


In [46]:
x1 = []
for i in range(len(x)):
    x1.append(np.zeros((max(lens_mfcc), 16)  ))
    x1[i][0:x[i].shape[0], 0:x[i].shape[1]]  =   x[i][0:x[i].shape[0], 0:x[i].shape[0]]

In [47]:
# x1[0:(x[i].shape[0])]

In [48]:
x1 = np.array(x1)

In [49]:
x1.shape

(150, 209, 16)

In [50]:
#C x покончено

In [51]:
lens_texts = []
for i in range(len(y)):
    lens_texts.append(len(y[i]))
    
print(max(lens_texts), min(lens_texts))


180 43


In [52]:
len(y[1])

84

In [53]:
y1 = []
for i in range(len(y)):
  #  print(x[i].shape)
    #try:
        y1.append(np.zeros(max(lens_texts)  ))
        y1[i][0:len(y[i])]  =   y[i][0:len(y[i])]
   # except:
    #    print(len(y[i]), len(y1[i]))

In [54]:
y1 = np.array(y1) 
y1.shape

(150, 180)

In [55]:
x1.shape

(150, 209, 16)

In [56]:
y_binary = to_categorical(y1)
y_binary.shape

(150, 180, 35)

In [57]:
x1.shape

(150, 209, 16)

In [58]:
y_binary.shape

(150, 180, 35)

In [59]:
# x_new=[]
# for j in range(x1.shape[0]):
#     k1=x1.shape[1]//y_binary.shape[1]
#     for i in range(y_binary.shape[1]-1):
#         for k in range(x1.shape[2]):
#             zxc=np.mean(x1 [j , i*k1:(i+1)*k1 ,k])
#             x_new.append(zxc)

In [60]:
# x_new1=np.array(x_new)
# x_new2=x_new1.reshape(2,-1,16)
# x_new2.shape

In [61]:
y_binary.shape

(150, 180, 35)

In [62]:
x1.shape

(150, 209, 16)

In [63]:
x_train, x_val, y_train, y_val = train_test_split(x1, y_binary, test_size=0.2, random_state=13)

In [64]:

model = Sequential()  
model.add(LSTM(128, return_sequences=True))  
model.add(LSTM(300, return_sequences=True))  
model.add(TimeDistributed(Dense(34 + 1)))
model.add(Activation("linear"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop") 

In [34]:
history = model.fit(x_train, y_train, epochs=7)

ValueError: Error when checking target: expected activation_1 to have shape (209, 35) but got array with shape (180, 35)

In [57]:
model.predict(x_new2).shape

(2, 102, 35)

In [62]:
for i in range(102):
    print(np.argmax(model.predict(x_new2)[0,i,:]))

16
16
16
16
16
16
16
34
1
1
1
1
1
1
21
21
21
21
21
21
6
6
6
14
14
14
10
10
10
10
10
10
10
10
10
10
10
10
10
10
34
10
10
10
10
6
6
6
6
6
10
18
18
18
18
18
18
34
34
34
34
34
34
34
1
1
1
1
1
1
13
13
13
13
13
34
10
10
10
10
10
10
10
10
10
10
14
14
14
14
14
6
6
6
6
16
16
16
16
16
16
16


In [55]:
        pred1 = model.predict(x_new2)
        #pred_class = (np.where(pred1 == np.amax(pred1))[1])
        pred_class = np.argmax(pred1)
        print(pred_class)

2775


In [83]:
    model2 = Sequential()
#     LSTM(units, activation = 'tanh', recurrent_activation= 'sigmoid', use_bias = true,
#          kernel_initializer = 'glorot_uniform', recurrent_initializer = 'orthogonal', 
#          bias_initializer = 'zeros', ... )
# Слои двунаправленных нейронов 
    model2.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2), input_shape=(None, 16)))
    model2.add(Bidirectional(LSTM(128, return_sequences=True, implementation=2)))
# Выходной слой 
    model2.add(TimeDistributed(Dense(len(inv_mapping) + 1, activation = 'relu') ))
    #model.a

In [84]:
model2.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, None, 256)         148480    
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 256)         394240    
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 35)          8995      
Total params: 551,715
Trainable params: 551,715
Non-trainable params: 0
_________________________________________________________________
None


In [87]:
history = model2.fit(x_train, y_train[:,:172,:], epochs=5, batch_size=50)

Epoch 1/5
120/120 [==============================] - 3s 23ms/step - loss: 7.8301 - accuracy: 0.2526
Epoch 2/5
120/120 [==============================] - 2s 20ms/step - loss: 4.9856 - accuracy: 0.5149
Epoch 3/5
120/120 [==============================] - 2s 20ms/step - loss: 4.6153 - accuracy: 0.5305
Epoch 4/5
120/120 [==============================] - 2s 20ms/step - loss: 4.4173 - accuracy: 0.5360
Epoch 5/5
120/120 [==============================] - 2s 20ms/step - loss: 4.2684 - accuracy: 0.5385


In [89]:
for i in range(102):
    print(
        np.argmax(model2.predict(x1)[0,i,:]))

0
0
10
10
10
15
15
15
15
15
15
15
15
15
15
15
34
34
34
34
34
34
34
34
34
34
34
12
12
12
12
12
12
12
12
34
34
34
34
34
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


KeyboardInterrupt: 

In [99]:
vocabulary['4']

KeyError: '4'

In [ ]:
model = Sequential()
model.add( LSTM( 128, input_shape=(180, 13), return_sequences=True))
# (bs, 45, 512)
model.add( LSTM( 128, return_sequences=True)) # SET HERE
# (bs, 512)
model.add( (Dense(35)))

In [ ]:
x1[:,0:180,:].shape

In [ ]:
history = model.fit(x_train, y_train, epochs=7)

In [ ]:
k = model.predict(x_train[0:1,0:180,:])

In [ ]:
k.shape

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [ ]:
import librosa
import librosa.display

In [ ]:
audio1 = audio.astype(np.float32)

In [ ]:
S = librosa.feature.melspectrogram(audio1 , sr=fs, n_mels=128)

# Convert to log scale (dB). We'll use the peak power (max) as reference.
log_S = librosa.power_to_db(S, ref=np.max)

plt.figure(figsize=(12, 4))
librosa.display.specshow(log_S, sr=fs, x_axis='time', y_axis='mel')
plt.title('Mel power spectrogram ')
plt.colorbar(format='%+02.0f dB')
plt.tight_layout()

In [ ]:
mfcc1 = librosa.feature.mfcc(S=log_S, n_mfcc=13)

In [ ]:
delta2_mfcc = librosa.feature.delta(mfcc1, order=2)

In [ ]:
(delta2_mfcc).T.shape

In [ ]:
features1 = mfcc(audio, samplerate=fs, lowfreq=50)

In [ ]:
features1.shape

In [ ]:
# Произведение вычислительных процессов для частей графа 
# Бесконенчый цикл, пока выполняется условие
# Сессия позволяет хранить значения графа и производить с ними вычисления 
with tf.Session(graph = graph) as session:

# Если нет чекпоинтов (chekpoint = None) 
# Инициализирум веса и биасы
        tf.global_variables_initializer().run()


    for epoch in range(last_epoch, num_epochs):

        for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_train1, y_train1, 100):
            feed_dict = {
                input_X: X_batch,
                labels: y_batch,

                seq_lens: seq_lens_batch
            }
            
            
# train_loss = session.run([ctc_loss])
# 
            train_loss, train_ler, train_decoded, true, _ = session.run([ctc_loss, ler, decoded[0], labels, train_op], feed_dict=feed_dict)
        print("train_decoded", train_loss)
        if epoch % epoch_save_step == 0 and epoch > 0:
                print("[i] SAVING snapshot %s" % snapshot)
#                 del tf.get_collection_ref ( ' LAYER_NAME_UIDS ' )[ 0 ]
                saver.save(session, "checkpoint1/" + snapshot + ".ckpt", epoch)

#         for X_batch, seq_lens_batch, y_batch, y_batch_orig in batch(X_test, y_test, 4):
#             feed_dict = {
#                 input_X: X_batch,
#                 labels: y_batch,
#                 seq_lens: seq_lens_batch
#             }
#             test_loss, test_ler, test_decoded, true = session.run([ctc_loss, ler, decoded[0], labels], feed_dict=feed_dict)
#         print(epoch, train_loss, train_ler)#,  test_loss, test_ler)
        ret = decode(train_decoded, inv_mapping)[:10]
        for i in range(len(ret)):
            print(str(ret[i])),
        print(time.ctime())
        decode1(y_batch_orig[0],inv_mapping)